## Esercizio 43

Un'industria manifatturiera può fabbricare $n$ tipi di prodotti che indichiamo genericamente con $N=\{1,\dots,n\}$ usando $m$ processi di produzione che avvengono mediante l'uso di $m$ macchine che indichiamo con $M=\{1,\dots,m\}$. Dopo aver dedotto il costo del materiale grezzo, ciascuna unità di prodotto dà un profitto $p_i \ \ i \in N$ (in euro).

Ciascuna unità di prodotto richiede un certo tempo di ciascuno degli $m$ processi; indichiamo con $t_{ji} \ \ i \in N, j \in M$ il tempo (in ore) di lavorazione di ciascuna macchina $j$ per ottenere una unità di ciascun prodotto finito $i$.

Inoltre, l'assemblaggio finale per ciascuna unità di ciascun prodotto richiede $u_i \ \ i \in N$ ore di lavoro di un operaio.
La fabbrica possiede una quantità prefissata di macchine di ciascun tipo con un limite sulle ore operative settimanali (capacità produttiva). Anche gli operai impiegati nell'assemblaggio sono in numero finito e ciascuno di essi ha un monteore prestabilito. Trovare la quantità che conviene produrre di ciascun prodotto per massimizzare il profitto totale.

I dati dell'istanza sono stati raccolti nel "file ex43.json".

\begin{align}
\max \quad & \sum\limits_{i \in N} p_i x_i& \\
s.t.\quad &  \sum\limits_{i \in N} t_{ji} x_i \leq T_j &j \in M\\
\quad & \sum\limits_{i \in N} u_i x_i \leq U &\\
\quad & x_i \geq 0 &i \in N
\end{align}


In [ ]:
# ALERT: execute this cell to prepare input data!
import requests
def download(link, nomeFile=None):
    if nomeFile == None:
        nomeFile = link.split('/')[-1]
    richiesta = requests.get(link)
    if richiesta.status_code == 200:
        with open(nomeFile, 'w') as file:
            file.write(richiesta.text)
            
download('https://tommasoadamo.it/data/ex43.json')

In [ ]:
# ALERT: execute this cell to install DOcplex! 
!pip install docplex cplex

In [ ]:
import json
import docplex.mp.model as cplex

with open('ex43.json', 'r') as f:
    dati = json.load(f)


n = len(dati['prodotti'])
m = len(dati['macchine'])

p = []     # lista dei profitti
t = dati['tempi_di_produzione']  #tempi di produzione
T = []     # capacità delle macchine 
u = []     # tempi assemblaggio prodotti
U = dati['operai']['quantita'] * dati['operai']['monteore']   # capacità assemblaggio

for prodotto in dati['prodotti']:
    p.append(prodotto['profitto'])
    u.append(prodotto['tempo_assemblaggio'])
    
for macchina in dati['macchine']:
    T.append(macchina['quantita']*macchina['capacita'])


N = list(range(n))
M = list(range(m))

with cplex.Model('ex43') as mdl:
    x = mdl.continuous_var_list(N, name='x')
    
    mdl.maximize(sum( p[i]*x[i]  for i in N))
    
    for j in M:
        mdl.add_constraint(sum( t[j][i]* x[i]  for i in N) <= T[j], 'cap_macchina_'+str(j))
    
    mdl.add_constraint(sum( u[i]*x[i] for i in N) <= U, 'cap_operai')
    
    mdl.print_information()
    
    s = mdl.solve()
    
    if s == None:
        print('No solution')
    else:
        print(s)